In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import pickle
from app.util.timer import Timer
from app.util.Differ import Differ
from main import YoloRuntimeTest

CPU input

In [2]:
args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_2.jpg", 
    "classes": "./app/weights/metadata.yaml", 
    "type": "image",
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_2.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

args_onnx_simplified = {
    "weights": "./app/weights/yolov9c-simplified.onnx", 
    "source": "./app/assets/sample_image_2.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_2.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False,
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

Initilize YOLO runtime test class

In [3]:
yolo_runtime_test = YoloRuntimeTest()

In [4]:
cpu_pytorch_image = yolo_runtime_test.ultralytics_run_image(args_pytorch)
print(cpu_pytorch_image)

[INFO] Initialize Model
[INFO] Inference Image


image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 480x640 5 cars, 6 traffic lights, 149.6ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 480x640 5 cars, 6 traffic lights, 149.6ms
Speed: 2.0ms preprocess, 149.6ms inference, 497.7ms postprocess per image at shape (1, 3, 480, 640)
Speed: 2.0ms preprocess, 149.6ms inference, 497.7ms postprocess per image at shape (1, 3, 480, 640)
Elapsed time: 2.2091 seconds
Class: car, Confidence: 0.93, Box: [859, 1689, 1509, 2235]
Class: car, Confidence: 0.92, Box: [155, 1659, 620, 2021]
Class: traffic light, Confidence: 0.91, Box: [366, 761, 480, 984]
Class: traffic light, Confidence: 0.82, Box: [0, 769, 45, 995]
Class: traffic light, Confidence: 0.80, Box: [1405, 1395, 1452, 1493]
Class: traffic light, Confidence: 0.78, Box: [1181, 1379, 1226, 1475]
Class: traffic light, Confidence: 0.76, Box: [991, 1367, 1

In [9]:
cpu_openvino_image = yolo_runtime_test.ultralytics_run_image(args_openvino)
print(cpu_openvino_image)

[INFO] Initialize Model
[INFO] Inference Image
Loading app\weights\yolov9c_openvino_model for OpenVINO inference...
Loading app\weights\yolov9c_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...
Using OpenVINO LATENCY mode for batch=1 inference...


image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 131.4ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 131.4ms
Speed: 9.3ms preprocess, 131.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Speed: 9.3ms preprocess, 131.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 1.9008 seconds
Class: car, Confidence: 0.93, Box: [859, 1690, 1509, 2235]
Class: car, Confidence: 0.92, Box: [155, 1659, 620, 2021]
Class: traffic light, Confidence: 0.91, Box: [366, 761, 480, 984]
Class: traffic light,

In [7]:
cpu_onnx_image = yolo_runtime_test.ultralytics_run_image(args_onnx)
print(cpu_onnx_image)

[INFO] Initialize Model
[INFO] Inference Image
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...


image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 195.3ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 195.3ms
Speed: 3.5ms preprocess, 195.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Speed: 3.5ms preprocess, 195.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 1.7779 seconds
Class: car, Confidence: 0.93, Box: [859, 1690, 1509, 2235]
Class: car, Confidence: 0.92, Box: [155, 1659, 620, 2021]
Class: traffic light, Confidence: 0.91, Box: [366, 761, 480, 984]
Class: traffic light, Confidence: 0.83, Box: [0, 769, 45, 995]
Class: traffic light, Confidence: 0.80, Box: [1405, 1395, 1452, 1494]
Class:

In [8]:
cpu_onnx_simplified_image = yolo_runtime_test.ultralytics_run_image(args_onnx_simplified)
print(cpu_onnx_simplified_image)

[INFO] Initialize Model
[INFO] Inference Image
Loading app\weights\yolov9c-simplified.onnx for ONNX Runtime inference...
Loading app\weights\yolov9c-simplified.onnx for ONNX Runtime inference...


image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 126.5ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_2.jpg: 640x640 5 cars, 6 traffic lights, 126.5ms
Speed: 3.0ms preprocess, 126.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Speed: 3.0ms preprocess, 126.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 1.7032 seconds
Class: car, Confidence: 0.93, Box: [859, 1690, 1509, 2235]
Class: car, Confidence: 0.92, Box: [155, 1659, 620, 2021]
Class: traffic light, Confidence: 0.91, Box: [366, 761, 480, 984]
Class: traffic light, Confidence: 0.83, Box: [0, 769, 45, 995]
Class: traffic light, Confidence: 0.80, Box: [1405, 13

In [12]:
cpu_onnx_runtime_image = yolo_runtime_test.onnxruntime_run_image(args_onnx)
print(cpu_onnx_runtime_image)

[INFO] Initialize Model
[INFO] Inference Image
Elapsed time: 0.1820 seconds
Class: car, Confidence: 0.94, Box: [856, 1686, 1511, 2235]
Class: traffic light, Confidence: 0.92, Box: [366, 761, 480, 985]
Class: car, Confidence: 0.91, Box: [154, 1658, 620, 2019]
Class: traffic light, Confidence: 0.83, Box: [0, 768, 44, 997]
Class: traffic light, Confidence: 0.82, Box: [991, 1367, 1042, 1466]
Class: traffic light, Confidence: 0.81, Box: [1403, 1394, 1452, 1491]
Class: traffic light, Confidence: 0.76, Box: [1180, 1380, 1226, 1475]
Class: car, Confidence: 0.76, Box: [2338, 1730, 2529, 1814]
Class: traffic light, Confidence: 0.65, Box: [1985, 1115, 2039, 1261]
Class: car, Confidence: 0.61, Box: [3640, 1692, 3969, 1803]
([['car', 0.9386053085327148, 856, 1686, 1511, 2235], ['traffic light', 0.9161757230758667, 366, 761, 480, 985], ['car', 0.9144351482391357, 154, 1658, 620, 2019], ['traffic light', 0.8307116031646729, 0, 768, 44, 997], ['traffic light', 0.8175124526023865, 991, 1367, 1042, 1466

In [13]:
cpu_onnx_simplified_runtime_image = yolo_runtime_test.onnxruntime_run_image(args_onnx_simplified)
print(cpu_onnx_simplified_runtime_image)

[INFO] Initialize Model
[INFO] Inference Image
Elapsed time: 0.1786 seconds
Class: car, Confidence: 0.94, Box: [856, 1686, 1511, 2235]
Class: traffic light, Confidence: 0.92, Box: [366, 761, 480, 985]
Class: car, Confidence: 0.91, Box: [154, 1658, 620, 2019]
Class: traffic light, Confidence: 0.83, Box: [0, 768, 44, 997]
Class: traffic light, Confidence: 0.82, Box: [991, 1367, 1042, 1466]
Class: traffic light, Confidence: 0.81, Box: [1403, 1394, 1452, 1491]
Class: traffic light, Confidence: 0.76, Box: [1180, 1380, 1226, 1475]
Class: car, Confidence: 0.76, Box: [2338, 1730, 2529, 1814]
Class: traffic light, Confidence: 0.65, Box: [1985, 1115, 2039, 1261]
Class: car, Confidence: 0.61, Box: [3640, 1692, 3969, 1803]
([['car', 0.9386053085327148, 856, 1686, 1511, 2235], ['traffic light', 0.9161757230758667, 366, 761, 480, 985], ['car', 0.9144351482391357, 154, 1658, 620, 2019], ['traffic light', 0.8307116031646729, 0, 768, 44, 997], ['traffic light', 0.8175124526023865, 991, 1367, 1042, 1466

Difference CPU

In [14]:
pd.set_option('display.expand_frame_repr', False)

def generate_difference_df(image1, image2, label):
    differ = Differ(np.array(image1), np.array(image2))
    result = differ.find_difference()
    return pd.DataFrame(result, columns=[label, "cpu conf_diff", "cpu box_diff (px)"])

df_pt_openvino = generate_difference_df(cpu_pytorch_image[0], cpu_openvino_image[0], "pt vs openvino+ultralytics")
df_pt_onnx = generate_difference_df(cpu_pytorch_image[0], cpu_onnx_image[0], "pt vs onnx+ultralytics")
df_pt_onnxruntime = generate_difference_df(cpu_pytorch_image[0], cpu_onnx_runtime_image[0], "pt vs onnxruntime")

df_combined = pd.concat([df_pt_openvino, df_pt_onnx, df_pt_onnxruntime], axis=1)

print(df_combined)

   pt vs openvino+ultralytics cpu conf_diff     cpu box_diff (px) pt vs onnx+ultralytics cpu conf_diff     cpu box_diff (px) pt vs onnxruntime cpu conf_diff           cpu box_diff (px)
0                         car        0.0025  [0.0, 1.0, 0.0, 0.0]                    car        0.0025  [0.0, 1.0, 0.0, 0.0]               car        0.0050        [3.0, 3.0, 2.0, 0.0]
1                         car        0.0006  [0.0, 0.0, 0.0, 0.0]                    car        0.0006  [0.0, 0.0, 0.0, 0.0]               car        0.0081        [1.0, 1.0, 0.0, 2.0]
2               traffic light        0.0018  [0.0, 0.0, 0.0, 0.0]          traffic light        0.0018  [0.0, 0.0, 0.0, 0.0]     traffic light        0.0093        [0.0, 0.0, 0.0, 1.0]
3               traffic light        0.0103  [0.0, 0.0, 0.0, 0.0]          traffic light        0.0103  [0.0, 0.0, 0.0, 0.0]     traffic light        0.0066        [0.0, 1.0, 1.0, 2.0]
4               traffic light        0.0001  [0.0, 0.0, 0.0, 1.0]          

Average CPU Time (10)

In [ ]:
result_time = []

In [ ]:
def run_inference_with_args(inference_func, args):
    def wrapper():
        return inference_func(args)
    return wrapper

def collect_execution_times(run_inference_func, args, iterations=10):
    execution_times = []
    for i in range(iterations):
        args["source"] = f"./app/assets/sample_image_{i}.jpg"
        wrapper_func = run_inference_with_args(run_inference_func, args)
        execution_time = wrapper_func()
        execution_times.append(execution_time[1] * 100)
    return execution_times

args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_0.jpg",
    "classes": "./app/weights/metadata.yaml", 
    "type": "image",
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "conf_threshold": 0.6, 
    "iou_threshold": 0.45, 
    "device": "cpu"
}

result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_pytorch))
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))
result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx))


In [ ]:
result_time = np.array(result_time)
df = pd.DataFrame(np.transpose(result_time), 
                  columns=["pytorch time cpu (ms)",
                           "openvino+ultralytics time cpu (ms)",
                           "onnx​+ultralytics time cpu (ms)", 
                           "onnx runtime time cpu (ms)"])
df.describe(percentiles=[.9, .95])

Save CPU result

In [ ]:
with open('./app/saved_pkl/cpu_df.pkl', 'wb') as f:
    pickle.dump(df, f)